In [1]:
import keras
import pandas as pd
import numpy as np
import cv2
import os
import imageio
import imgaug.augmenters as iaa
import imgaug as ia
import math
import random
from keras.applications.resnet_v2 import preprocess_input
from sklearn.model_selection import train_test_split
import tensorflow as tf

**Preprocess the images and store them as labled data in csv file.**

In [ ]:
def video_to_frames(video):
    vidcap = cv2.VideoCapture(video)
    ids = set()
    while len(ids) <= 4:
      ids.add(random.randint(1, vidcap.get(cv2.CAP_PROP_FRAME_COUNT) - 1))

    ImageFrames = []
    for id in ids:
      vidcap.set(cv2.CAP_PROP_POS_FRAMES, id - 1)
      success, image = vidcap.read()

      if success:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = preprocess_input(image)
        ImageFrames.append(image)
      else:
        print('Failure!!!')

    vidcap.release()

    return ImageFrames

In [ ]:
from pathlib import Path

def get_all_file_paths(directory):
    return [str(file) for file in Path(directory).rglob('*') if file.is_file()]

# NonViolence
paths_nonvi = []
paths_vi = []
for i in range(1, 6):
  directory_nonvi = f'/content/drive/MyDrive/Violence Detection/data/violent_flow/movies/{i}/NonViolence'
  directory_vi = f'/content/drive/MyDrive/Violence Detection/data/violent_flow/movies/{i}/Violence'
  paths_nonvi.extend(get_all_file_paths(directory_nonvi))
  paths_vi.extend(get_all_file_paths(directory_vi))

In [ ]:
violence_frames = []
non_violence_frames = []
for path in paths_vi:
  violence_frames.extend(video_to_frames(path))
for path in paths_nonvi:
  non_violence_frames.extend(video_to_frames(path))

In [ ]:
violence_dataframe = pd.DataFrame({'image': violence_frames, 'y': np.ones(615, dtype = int)})
non_violence_dataframe = pd.DataFrame({'image': non_violence_frames, 'y': np.zeros(615, dtype = int)})
data = pd.concat([violence_dataframe, non_violence_dataframe], ignore_index = True)

In [ ]:
shuffled_data = data.sample(frac=1).reset_index(drop=True)

In [ ]:
shuffled_data_numpy = shuffled_data.to_numpy()

In [ ]:
np.save('/content/drive/MyDrive/Violence Detection/data.npy', shuffled_data_numpy)

**Transfer train the ResNet model**

In [2]:
data = np.load('/content/drive/MyDrive/Violence Detection/data.npy', allow_pickle = True)

In [3]:
base_model = keras.applications.ResNet152V2(
    include_top=False,
    weights="imagenet",
    classifier_activation="softmax",
    name="resnet152v2",
)

234545216/234545216 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [4]:
base_model.trainable = False

In [5]:
inputs = keras.layers.Input(shape = (240, 320, 3))

In [6]:
x = base_model(inputs, training = False)

In [7]:
y = keras.layers.GlobalAveragePooling2D()(x)

In [8]:
outputs = keras.layers.Dense(1, activation = 'sigmoid')(y)

In [9]:
model = keras.Model(inputs, outputs)

In [10]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 240, 320, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ resnet152v2 (Functional)             │ (None, 8, 10, 2048)         │      58,331,648 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 2048)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │           2,049 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 58,333,697 (222.53 MB)

 Trainable params: 2,049 (8.00 KB)

 Non-trainable params: 58,331,648 (222.52 MB)

In [11]:
x = data[:, 0]
y = data[:, 1]

In [12]:
x_reshaped = np.array([row for row in x])

In [13]:
x_reshaped.shape

(1230, 240, 320, 3)

In [14]:
x_reshaped_tensor = tf.convert_to_tensor(x_reshaped)

In [15]:
y_reshaped = np.array([row for row in y])

In [16]:
y_reshaped_tensor = tf.convert_to_tensor(y_reshaped)

In [17]:
x_train, x_test, y_train, y_test = train_test_split(x_reshaped, y_reshaped, test_size = 0.2, random_state = 42)

In [18]:
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[keras.metrics.BinaryAccuracy()],
)

epochs = 6
model.fit(x_train, y_train, epochs=epochs, batch_size = 32)

Epoch 1/6


/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/nn.py:707: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


31/31 ━━━━━━━━━━━━━━━━━━━━ 44s 547ms/step - binary_accuracy: 0.7266 - loss: 0.5487
Epoch 2/6
31/31 ━━━━━━━━━━━━━━━━━━━━ 21s 254ms/step - binary_accuracy: 0.9037 - loss: 0.2555
Epoch 3/6
31/31 ━━━━━━━━━━━━━━━━━━━━ 10s 256ms/step - binary_accuracy: 0.9321 - loss: 0.2077
Epoch 4/6
31/31 ━━━━━━━━━━━━━━━━━━━━ 11s 266ms/step - binary_accuracy: 0.9596 - loss: 0.1467
Epoch 5/6
31/31 ━━━━━━━━━━━━━━━━━━━━ 11s 286ms/step - binary_accuracy: 0.9742 - loss: 0.1214
Epoch 6/6
31/31 ━━━━━━━━━━━━━━━━━━━━ 9s 271ms/step - binary_accuracy: 0.9879 - loss: 0.1020


In [20]:
model.save('/content/drive/MyDrive/Violence Detection/model.keras')

In [21]:
model.evaluate(x_test, y_test, batch_size = 32)

8/8 ━━━━━━━━━━━━━━━━━━━━ 19s 1s/step - binary_accuracy: 0.9706 - loss: 0.1319


[0.13625501096248627, 0.9634146094322205]